In [ ]:
import pandas as pd

data = pd.read_csv('dataset_NYC.txt', encoding="unicode_escape", names=['use_ID', 'loc_ID', 'loc_cat_ID', 'loc_cat_name', 'latitude', 'longitude', 'time_offset', 'time'], sep='\t')
data_exact = data[['use_ID', 'loc_ID', 'loc_cat_name', 'latitude', 'longitude', 'time_offset', 'time']]
time = pd.to_datetime(data_exact['time'])
time_offset = data_exact['time_offset'].values
def local_time(x, y):
    return x + pd.Timedelta(minutes = y)
time_local = list(map(local_time, time, time_offset))
data_exact['time'] = pd.DataFrame(time_local)

In [2]:
#过滤数据
def filter(data):
    dm = data[['use_ID', 'loc_ID']]
    user_cnt = dm.groupby(by=['use_ID'], as_index=False).count()
    user_list = list(user_cnt[user_cnt['loc_ID'] >= 10]['use_ID'])#过滤掉签到次数少于10次的用户
    ds = data[data['use_ID'].isin(user_list)]
    loc_cnt = ds.groupby(by=['loc_ID'], as_index=False).count()
    loc_list = list(loc_cnt[loc_cnt['use_ID'] >= 10]['loc_ID'])#过滤掉签到人数少于10人次的位置
    data_filtered = ds[ds['loc_ID'].isin(loc_list)]
    return data_filtered
data_filtered = filter(data_exact)
user_cnt = data_filtered.groupby(by=['use_ID'], as_index=False).count()
control = True
while control:
    if (any(user_cnt['loc_ID'] < 10)):
        data_filtered = filter(data_filtered)
        user_cnt = data_filtered.groupby(by=['use_ID'], as_index=False).count()
    else:
        control = False
data_filtered.sort_values(by='time', inplace=True)#按时间排序
data_filtered.index = range(len(data_filtered))

In [3]:
new_cat = pd.read_csv('FS_NYC_CAT_NAME_N.csv', encoding="unicode_escape", names=['loc_cat_name', 'Quantity', 'new_cat_name'], sep=',', header=0)
cat_map = new_cat[['loc_cat_name', 'new_cat_name']]
cat_map.iloc[52][0] = 'Café'
origin_cat_name = data_filtered['loc_cat_name']
cat_map_dict = {cat_map.iloc[i][0]:cat_map.iloc[i][1] for i in range(len(cat_map))}
loc_cat_new_name = pd.Series(map(lambda x: cat_map_dict[x], origin_cat_name))
data_filtered.insert(7, 'loc_cat_new_name', loc_cat_new_name)

In [4]:
#处理一个loc_ID有多个语义类别的情况，具体：位置A对应b和c两种类别，类别b包含有100个位置，类别c包含有50个位置，则将位置A设为类别b
loc_and_cat = data_filtered[['loc_ID', 'loc_cat_new_name']].drop_duplicates()
cat_include_loc = loc_and_cat.groupby(by='loc_cat_new_name').count()#统计各类别下包含位置的数量
cat_include_loc.columns = ['num']
cat_include_loc_num = {cat_include_loc.index[i]:cat_include_loc['num'][i] for i in range(len(cat_include_loc))}

In [5]:
#筛选一个位置对应多个类别的情况
loc_has_cat = loc_and_cat.groupby(by='loc_ID').count()
multicat_cat_loc = loc_has_cat[loc_has_cat['loc_cat_new_name']>=2]#index:loc_ID
#获取有多个语义的位置及其对应的语义
multi_loc_item = loc_and_cat[loc_and_cat['loc_ID'].isin(multicat_cat_loc.index)]
multi_loc_item = multi_loc_item.sort_values(by='loc_ID')
multi_loc_item.index = range(len(multi_loc_item))
mul_loc_ID = multicat_cat_loc.index.sort_values()

In [16]:
#处理一个loc_ID有多个语义类别的情况，具体：位置A对应b和c两种类别，类别b包含有100个位置，类别c包含有50个位置，则将位置A设为类别c(减小类别不平衡)
unified_cat = []
for i, loc_id in enumerate(mul_loc_ID):
    name = list(multi_loc_item[multi_loc_item['loc_ID'] == loc_id]['loc_cat_new_name'])
    if len(name) == 2:
        unified_name = name[0] if cat_include_loc_num[name[0]] < cat_include_loc_num[name[1]] else name[1]
    else:
        tmp_cat = name[0]
        for i in range(len(name)-1):
            tmp_cat = tmp_cat if cat_include_loc_num[tmp_cat] < cat_include_loc_num[name[i+1]] else name[i+1]
        unified_name = tmp_cat
    unified = [unified_name] * len(name)
    # unified_cat.append(unified)
    unified_cat = unified_cat + unified
multi_loc_item.insert(1, 'cat_name', pd.Series(unified_cat))
loc_new_cat = multi_loc_item[['loc_ID', 'cat_name']].drop_duplicates()
loc_new_cat.index = range(len(loc_new_cat))

In [ ]:
#更新过滤数据中的类别信息（对应多个语义的）
data_loc_m_to_o = pd.DataFrame()
data_loc_m_c = data_filtered[data_filtered['loc_ID'].isin(loc_new_cat['loc_ID'])]
for i in range(len(loc_new_cat)):
    tmp = data_loc_m_c[data_loc_m_c['loc_ID'].isin([loc_new_cat['loc_ID'][i]])]
    tmp['loc_cat_new_name'] = loc_new_cat['cat_name'][i]
    data_loc_m_to_o = pd.concat([data_loc_m_to_o, tmp])
data_loc_o_c = data_filtered[~data_filtered['loc_ID'].isin(loc_new_cat['loc_ID'])]
data_processed = pd.concat([data_loc_o_c, data_loc_m_to_o])
data_processed.sort_values(by='time', inplace=True)#按时间排序
data_processed.index = range(len(data_processed))

In [65]:
data_processed = data_processed[['use_ID', 'loc_ID', 'loc_cat_new_name', 'latitude', 'longitude', 'time']]
data_processed.to_csv('FS_NYC_Cat.csv', sep=',', index=False, header=True)#原始数据过滤后添加新的类别信息

In [1]:
#数据集划分：按时序，8:1:1（细节：在分割点处，保证一天的完整轨迹）
import pandas as pd
data = pd.read_csv('FS_NYC_Cat.csv', names=['use_ID', 'loc_ID', 'loc_cat_new_name', 'latitude', 'longitude', 'time'], sep=',', header=0)
data['time'] = pd.to_datetime(data['time'])

In [2]:
train = data.head(int(0.8*147938)-37)#-37：保证邻近分割时间点完整一天的签到记录归纳到训练集或验证集
tmp = data.head(int(0.9*147938-77))
valid = tmp.tail(len(tmp)-len(train))
test = data.tail(len(data)-len(tmp))

In [ ]:
#将在训练集中没有而在验证集中出现的用户ID或者位置ID的相关签到记录排除
train_user = set(train['use_ID'])
val_user = set(valid['use_ID'])
test_user = set(test['use_ID'])
train_loc = set(train['loc_ID'])
val_loc = set(valid['loc_ID'])
test_loc = set(test['loc_ID'])
val_g = valid.groupby(by='use_ID')
val_train_user = val_user - train_user
for use_id, valg in val_g:
    if use_id in val_train_user:
        valid.drop(valg.index, inplace=True)
val_g = valid.groupby(by='loc_ID')
val_train_loc = val_loc - train_loc
for loc_id, valg in val_g:
    if loc_id in val_train_loc:
        valid.drop(valg.index, inplace=True)
        
#将在训练集中没有而在测试集中出现的用户ID或者位置ID的相关签到记录排除
test_g = test.groupby(by='use_ID')
test_train_user = test_user - train_user
for use_id, valg in test_g:
    if use_id in test_train_user:
        test.drop(valg.index, inplace=True)
test_g = test.groupby(by='loc_ID')
test_train_loc = test_loc - train_loc
for loc_id, valg in test_g:
    if loc_id in test_train_loc:
        test.drop(valg.index, inplace=True)
train.to_csv('train.csv', sep=',', index=False, header=True)
valid.to_csv('valid.csv', sep=',', index=False, header=True)
test.to_csv('test.csv', sep=',', index=False, header=True)

In [4]:
#处理后，验证集和测试集中出现的用户和位置均在训练集中出现过
#可以对训练集中的用户、位置和语义类别重新编号
import pandas as pd
data = pd.read_csv('train.csv', names=['use_ID', 'loc_ID', 'loc_cat_new_name', 'latitude', 'longitude', 'time'], sep=',', header=0)
user = list(set(data['use_ID']))
user_id = {}
for i, user in enumerate(user):
    user_id[user] = i  #用户新编号
user_id = pd.DataFrame(list(user_id.items()))
user_id.columns = ['old_id', 'new_id']
user_id.to_csv('user_id.csv', sep=',', index=False, header=True)

In [ ]:
#添加类别ID和具体位置在类别内的ID
import pandas as pd
data = pd.read_csv('train.csv', names=['use_ID', 'loc_ID', 'loc_cat_new_name', 'latitude', 'longitude', 'time'], sep=',', header=0)
category = data['loc_cat_new_name'].drop_duplicates()
cat_id = {}
for i, cat in enumerate(category):
    cat_id[cat] = i  #语义类别编号
#添加类别ID
data.insert(6, 'cat_id', -1)
for i in range(len(data)):
    data['cat_id'][i] = cat_id[data['loc_cat_new_name'][i]]
# cat_id = pd.DataFrame(list(cat_id.items()))
# cat_id.columns = ['cat_name', 'cat_id']
# cat_id.to_csv('category_id.csv', sep=',', index=False, header=True)

In [13]:
#添加位置在类别的ID
loc_group1 = data[['loc_ID', 'loc_cat_new_name', 'cat_id']].drop_duplicates()
loc_group2 = data[['loc_ID', 'latitude', 'longitude']].drop_duplicates()
loc_add_catin = pd.DataFrame()
loc_group1.insert(3, 'loc_catin_id', -1)
loc_g1 = loc_group1.groupby(by='cat_id')
for i, group in loc_g1:
    group['loc_catin_id'] = range(len(group))
    loc_add_catin = pd.concat([loc_add_catin, group])
loc_add_catin.sort_values(by='loc_ID', inplace=True)
loc_add_catin.index = range(len(loc_add_catin))
#处理同一个位置ID,具有多个不同经纬度的情况
loc_singlecoor = pd.DataFrame()
loc_g2 = loc_group2.groupby(by='loc_ID')
for i, group in loc_g2:
    if len(group) == 1:
        loc_singlecoor = pd.concat([loc_singlecoor, group])
    else:
        assemble = pd.DataFrame([i, group['latitude'].mean(), group['longitude'].mean()]).T
        assemble.columns = ['loc_ID', 'latitude', 'longitude']
        loc_singlecoor = pd.concat([loc_singlecoor, assemble])
loc_singlecoor.index = range(len(loc_singlecoor))
loc = pd.concat([loc_add_catin, loc_singlecoor[['latitude', 'longitude']]], axis=1)
loc['loc_new_ID'] = loc.index
loc.to_csv('loc.csv', sep=',', index=False, header=True)

In [ ]:
#将训练集、验证集、测试集中的用户ID和位置ID替换成新ID，同时添加类别ID和位置在类别内的ID
import pandas as pd
data = pd.read_csv('test.csv', names=['use_ID', 'loc_ID', 'loc_cat_new_name', 'latitude', 'longitude', 'time'], sep=',', header=0)
loc = pd.read_csv('loc.csv', names=['loc_ID', 'loc_cat_new_name', 'cat_id', 'loc_catin_id', 'latitude', 'longitude', 'loc_new_ID'], sep=',', header=0)
user = pd.read_csv('user_id.csv', names=['old_id', 'new_id'], sep=',', header=0)
user_id = {}
loc_id = {}
for i in range(len(user)):
    user_id[user['old_id'][i]] = user['new_id'][i]
for i in range(len(loc)):
    loc_id[loc['loc_ID'][i]] = loc.loc[i, ['loc_new_ID', 'cat_id', 'loc_catin_id']]
    # loc_id[loc['loc_ID'][i]] = loc['loc_new_ID'][i]
data['user_new_ID'] = -1
data['loc_new_ID'] = -1
data['cat_id'] = -1
data['loc_catin_id'] = -1
for i in range(len(data)):
    data['user_new_ID'][i] = user_id[data['use_ID'][i]]  
    data['loc_new_ID'][i] = loc_id[data['loc_ID'][i]][0]
    data['cat_id'][i] = loc_id[data['loc_ID'][i]][1]
    data['loc_catin_id'][i] = loc_id[data['loc_ID'][i]][2]
train_data = data[['user_new_ID', 'loc_new_ID', 'time', 'cat_id', 'loc_catin_id']]
train_data.to_csv('NYC_test.csv', sep=',', index=False, header=True)

In [ ]:
#将训练集、验证集、测试集签到数据按用户，连续两次签到间隔超过24小时，拆分为不同的轨迹
import pandas as pd
data = pd.read_csv('NYC_test.csv', names=['user_new_ID', 'loc_new_ID', 'time', 'cat_id', 'loc_catin_id'], sep=',', header=0)
data.sort_values(['user_new_ID','time'], ascending=[True, True], inplace=True)
data.index = range(len(data))
#生成每条签到记录的轨迹号
data['time'] = pd.to_datetime(data['time'])
data_g = data.groupby(by='user_new_ID')
data_traj = pd.DataFrame()
for user, datag in data_g:
    traj = 0
    datag['traj_num'] = -1
    datag.index = range(len(datag))
    f = 0
    for i in range(len(datag)-1):
        deltatime = datag['time'][i+1] - datag['time'][i]
        if deltatime.days >= 1:
            datag['traj_num'][f:i+1] = traj
            f = i + 1
            traj += 1
    datag['traj_num'][f:] = traj
    data_traj = pd.concat((data_traj, datag))
data_traj.index = range(len(data_traj))
#生成每条签到记录所在的星期几名字
week = [data_traj['time'][i].isoweekday() for i in range(len(data_traj))]
data_traj['day_of_week'] = week
#生成每条签到记录的时间片
tm = data_traj['time']
tms = list(map(lambda x: x.hour * 2 + 2 if x.minute >=30 else x.hour * 2 + 1, tm))
data_traj['timeslot'] = pd.Series(tms)
data_traj.to_csv('test_traj.csv', sep=',', index=False, header=True)

In [ ]:
# data_traj_g = data_traj.groupby(by=['user_new_ID'])
# traj_all = []
# # traj_all = dict()
# for i, u_traj in data_traj_g:
#     u_traj_g = u_traj.groupby(by=['traj_num'])
#     traj_src = dict()
#     traj_pred = dict()
#     for n, d_traj in u_traj_g:
#         d_t = d_traj[['loc_new_ID', 'timeslot', 'day_of_week', 'cat_id', 'loc_catin_id']]
#         if len(d_t) > 1: #排除只有一个轨迹点的轨迹
#             traj_points = [(d_t.iloc[j][0], d_t.iloc[j][1], d_t.iloc[j][2], d_t.iloc[j][3], d_t.iloc[j][4]) for j in range(len(d_t))]
#         # if len(traj_points) > 1:
#             traj_src[n] = traj_points[:-1]
#             traj_pred[n] = traj_points[-1]
#     # traj_all[i] = {'forward': traj_src, 'pred': traj_pred}
#     if len(traj_pred) > 0:
#         traj_all.append({i: {'forward': traj_src, 'pred': traj_pred}})

In [ ]:
#提取轨迹内的转移边，构造全局转移图 
import pandas as pd
data = pd.read_csv('TKY_train.csv', names=['user_new_ID', 'loc_new_ID', 'time'], sep=',', header=0)
data.sort_values(['user_new_ID','time'], ascending=[True, True], inplace=True)
data.index = range(len(data))
data['time'] = pd.to_datetime(data['time'])
data_g = data.groupby(by='user_new_ID')
data_traj = pd.DataFrame()
#提取用户的各条轨迹
for user, datag in data_g:
    traj = 0
    datag['traj_num'] = -1
    datag.index = range(len(datag))
    f = 0
    for i in range(len(datag)-1):
        deltatime = datag['time'][i+1] - datag['time'][i]
        if deltatime.days >= 1:
            datag['traj_num'][f:i+1] = traj
            f = i + 1
            traj += 1
    datag['traj_num'][f:] = traj
    data_traj = pd.concat((data_traj, datag))
data_traj.index = range(len(data_traj))

#提取轨迹内的转移边：共82965条转移连边（训练集）
data_traj_g = data_traj.groupby(by=['user_new_ID', 'traj_num'])
edge_trans = []
for i, traj in data_traj_g:
    traj.index = range(len(traj))
    trans_e = [(traj['loc_new_ID'][j], traj['loc_new_ID'][j+1]) for j in range(len(traj) - 1)]
    edge_trans += trans_e

In [2]:
#统计转移边及其频次 有向边：src->dst：频次
from collections import Counter
edge_trans_cnt = Counter(edge_trans)#最大频次为
global_tran_e = dict(edge_trans_cnt)
global_tran_edge = pd.DataFrame(global_tran_e.keys())
global_tran_edge.columns = ['src', 'dst']
global_tran_edge['freq'] = global_tran_e.values()
gtedge_g = global_tran_edge.groupby(by='dst')
tran_edge = pd.DataFrame()
for dst, e in gtedge_g:
    e.index = range(len(e))
    total_freq = e['freq'].sum()
    e['weight'] = e['freq'] / total_freq
    tran_edge = pd.concat((tran_edge, e))
tran_edge.to_csv('tran_edge.csv', sep=',', index=False, header=True)

In [ ]:
#构建位置的地理空间连边  haversine距离小于1000m
import pandas as pd
import transbigdata as tbd
import geohash as gh
from haversine import haversine
data = pd.read_csv('loc.csv', names=['loc_ID', 'latitude', 'longitude', 'loc_new_ID'], sep=',', header=0)
loc = data[['loc_new_ID', 'latitude', 'longitude']]
geohash = tbd.geohash_encode(loc['longitude'], loc['latitude'], precision=5)#precision:5(4.89*4.89km),6(1.22*0.61km)
geo_edge = []
for i in range(len(loc)):
    neigh = gh.neighbors(geohash[i])
    for k in neigh:
        loc_geo = geohash[geohash.values == neigh[k]]
        if len(loc_geo) == 0:
            continue
        else:
            # for j in loc_geo.index:
            #     dist = haversine((loc['latitude'][i], loc['longitude'][i]), (loc['latitude'][j], loc['longitude'][j]), unit='m')
            #     if dist <= 1000:#超参：距离小于1000m的两个位置建立连边
            #         geo_edge +=[(i, j)]
            geo_edge += [(i, j) for j in loc_geo.index if haversine((loc['latitude'][i], loc['longitude'][i]), (loc['latitude'][j], loc['longitude'][j]), unit='m')<=1000]
geo_edge = pd.DataFrame(geo_edge)
geo_edge.columns = ['src', 'dst']
geo_edge.to_csv('geo_edge.csv', sep=',', index=False, header=True)

In [1]:
#构建同类别连边，目的：建模层次约束关系
#统计32个语义类别下位置的最高数量
import pandas as pd
data = pd.read_csv('loc.csv', names=['loc_ID', 'loc_cat_new_name', 'cat_id', 'loc_catin_id','latitude', 'longitude', 'loc_new_ID'], sep=',', header=0)
data.sort_values(by=['cat_id', 'loc_catin_id'], ascending=[True, True], inplace=True)
data.index = range(len(data))
data_g = data.groupby(by='cat_id')
cat_edge = []
for i, group in data_g:
    loc_new_id = group['loc_new_ID']
    loc_new_id.index = range(len(loc_new_id))
    for j in range(len(loc_new_id)):
        cat_edge +=([(loc_new_id[j], loc_new_id[k]) for k in range(len(loc_new_id))])
cat_edge = pd.DataFrame(cat_edge)
cat_edge.columns = ['src', 'dst']
cat_edge.to_csv('cat_edge.csv', sep=',', index=False, header=True)

In [4]:
import pandas as pd
import numpy as np
import pickle
traj_data = pd.read_csv('train_traj.csv', names=['user_new_ID','loc_new_ID','time', 'cat_id', 'loc_catin_id', 'traj_num','day_of_week','timeslot'], sep=',', header=0)
data_traj_g = traj_data.groupby(by=['user_new_ID', 'traj_num'])
traj_f = []
traj_p = []
traj_user = []
traj_user_traj_n = []
for i, traj in data_traj_g:
    d_t = np.array(traj[['loc_new_ID', 'timeslot', 'day_of_week', 'cat_id', 'loc_catin_id']])
    if len(d_t) > 1:#剔除只有一条签到记录的轨迹
        traj_tmp = [tuple(d_t[i]) for i in range(len(d_t))]
        traj_f.append(traj_tmp[:-1])
        # traj_p.append(traj_tmp[-1])#只提取最后一次
        traj_p.append(traj_tmp[1:])#从第2个签到记录开到最后
        traj_user.append(i[0])
        traj_user_traj_n.append(i[1])
file = open('train_forward.pickle','wb')
pickle.dump(traj_f,file)
file.close()
file = open('train_labels.pickle','wb')
pickle.dump(traj_p,file)
file.close()
file = open('train_user.pickle','wb')
pickle.dump(traj_user,file)
file.close()
# traj_p = pd.DataFrame(traj_p)
# traj_p.columns = ['loc_new_ID', 'timeslot', 'day_of_week']
# traj_user = pd.DataFrame(traj_user)
# traj_user.columns = ['user_new_ID']
# traj_pred = pd.concat((traj_user, traj_p), 1)
# traj_pred.to_csv('valid_pred.csv', sep=',', index=False, header=True)

In [ ]:
#统计轨迹平均长度
import pandas as pd
import numpy as np
train_data = pd.read_csv('train_traj.csv', names=['user_new_ID','loc_new_ID','time','traj_num','day_of_week','timeslot'], sep=',', header=0)
valid_data = pd.read_csv('valid_traj.csv', names=['user_new_ID','loc_new_ID','time','traj_num','day_of_week','timeslot'], sep=',', header=0)
test_data = pd.read_csv('test_traj.csv', names=['user_new_ID','loc_new_ID','time','traj_num','day_of_week','timeslot'], sep=',', header=0)
train_data_g = train_data.groupby(by=['user_new_ID', 'traj_num'])
valid_data_g = valid_data.groupby(by=['user_new_ID', 'traj_num'])
test_data_g = test_data.groupby(by=['user_new_ID', 'traj_num'])
train_data_len = []
valid_data_len = []
test_data_len = []
for i, traj in train_data_g:
    d_t = np.array(traj[['loc_new_ID', 'timeslot', 'day_of_week']])
    if len(d_t) > 1:
        train_data_len.append(len(d_t))
for i, traj in valid_data_g:
    d_t = np.array(traj[['loc_new_ID', 'timeslot', 'day_of_week']])
    if len(d_t) > 1:
        valid_data_len.append(len(d_t))
for i, traj in test_data_g:
    d_t = np.array(traj[['loc_new_ID', 'timeslot', 'day_of_week']])
    if len(d_t) > 1:
        test_data_len.append(len(d_t))
#平均轨迹长度为5.2
mean_len = (sum(train_data_len)+sum(valid_data_len)+sum(test_data_len))/(len(train_data_len)+len(valid_data_len)+len(test_data_len))
print(mean_len)

In [ ]:
import pandas as pd
import numpy as np
loc = pd.read_csv('loc.csv', names=['loc_ID', 'loc_cat_new_name', 'cat_id', 'loc_catin_id', 'latitude', 'longitude', 'loc_new_ID'], sep=',', header=0)
loc_cat = loc[['loc_new_ID', 'cat_id']]
loc_cat.sort_values(by='loc_new_ID', ascending=True, inplace=True)

In [2]:
loc_map_cat = {}
for i in range(len(loc_cat)):
    loc_map_cat[i] = loc_cat['cat_id'][i]

In [4]:
import torch
a = [1, 2, 3]
b = [2, 3, 4]
a = torch.tensor(a)
b = torch.tensor(b)
torch.matmul(a,b)

tensor(20)